In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
import itertools
import random
import warnings
import numpy as np
import cv2
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

warnings.simplefilter(action='ignore', category=FutureWarning)

# Sign Language Model

### Setup

#### 1) Create a dictionary of numbers to letters so that we can decode the message at the end of training the model

In [ ]:
word_dict = {}

letter_idx = 97
for i in range(26):
    word_dict[i] = chr(letter_idx)
    letter_idx += 1

print(word_dict)

#### 2) Initialize the train and test batches with the ImageDataGenerator object 

In [ ]:
train_path = './data/train'
test_path = './data/test'

# batch_size = How many do we want to predict? chooses images at random 
batch_size = 20
class_mode = 'categorical'
target_size = (64, 64)
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=train_path, target_size=target_size, class_mode=class_mode, batch_size=batch_size,shuffle=True)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=test_path, target_size=target_size, class_mode=class_mode, batch_size=batch_size, shuffle=True)

imgs, labels = next(train_batches)

print("Batched train and test images...")

### Build the Model
#### 1) Create the model that correctly classifies each of the letters 

In [ ]:
model = Sequential()

print("model 1")
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(64,64,3)))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

print("First convolution...")

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

print("Second convolution...")

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding = 'valid'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

print("Convolutions done")
print("Flattening...")

model.add(Flatten())

model.add(Dense(64,activation ="relu"))
model.add(Dense(128,activation ="relu"))
model.add(Dense(128,activation ="relu"))
model.add(Dense(5,activation ="softmax"))
# Softmax must be equivalent to the number of classifications (26 for full alphabet)

#### 2) We compile and define the loss functions here so that we can create a fitted model in the next step

In [ ]:
print("Compiling with ADAM model...")

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.0001)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

print("Compiling with SGD model...")

model.compile(optimizer=SGD(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.0005)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

#### 3) Fit and save the model using the loss functions defined above 

In [ ]:
print("Fitting model...")
fitted_model = model.fit(train_batches, epochs=9, callbacks=[reduce_lr, early_stop],  validation_data = test_batches)#, checkpoint])

imgs, labels = next(test_batches) # For getting next batch of imgs...

scores = model.evaluate(imgs, labels, verbose=0)
print(f'{model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

print("Saving model...")
model.save('best_model_dataflair3.h5')

### Classify

#### 1) Load the model

In [ ]:
model = keras.models.load_model(r"best_model_dataflair3.h5")

imgs, labels = next(test_batches)

scores = model.evaluate(imgs, labels, verbose=0)
print(f'{model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

model.summary()

scores #[loss, accuracy] on test data...
model.metrics_names

#### 2) Test the model 

In [20]:


predictions = model.predict(imgs, verbose=0)
print("predictions:", predictions)

print("Test our predictions using a batch size of:", batch_size)
pred = []
for ind, i in enumerate(predictions):
    print(word_dict[np.argmax(i)], end='   ')
    pred.append(word_dict[np.argmax(i)])

actual = []
for i in labels:
    print(word_dict[np.argmax(i)], end='   ')
    actual.append(word_dict[np.argmax(i)])

tot = 0
correct = 0
for i in range(len(labels)):
    if labels[i] == pred[i]:
        correct += 1
    tot += 1

print("Accuracy:", 1.0 * correct / total)
print("Actual labels:", labels)
print("Predictions:  ", predictions)

Found 1385 images belonging to 26 classes.
Found 120 images belonging to 26 classes.
Batched train and test images...
(5, 64, 64, 3)
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]]
model 1
First convolution...
Second convolution...
Convolutions done
Flattening...
Compiling...
Compiling 2...
Fitting model...
Epoch 1/9


InvalidArgumentError:  logits and labels must be broadcastable: logits_size=[5,5] labels_size=[5,26]
	 [[node categorical_crossentropy/softmax_cross_entropy_with_logits
 (defined at /Users/meghamattikalli/opt/anaconda3/lib/python3.8/site-packages/keras/backend.py:5009)
]] [Op:__inference_train_function_99419]

Errors may have originated from an input operation.
Input Source operations connected to node categorical_crossentropy/softmax_cross_entropy_with_logits:
In[0] categorical_crossentropy/softmax_cross_entropy_with_logits/Reshape:	
In[1] categorical_crossentropy/softmax_cross_entropy_with_logits/Reshape_1:

Operation defined at: (most recent call last)
>>>   File "/Users/meghamattikalli/opt/anaconda3/lib/python3.8/runpy.py", line 194, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "/Users/meghamattikalli/opt/anaconda3/lib/python3.8/runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "/Users/meghamattikalli/opt/a